# Логистическая регрессия

Особенность: возможность оценивания вероятностей классов

### Реализация градиентного спуска

In [20]:
import pandas
import math
from sklearn.metrics import roc_auc_score
import sympy
from IPython.display import Image
from IPython.core.display import HTML 

#Загрузите данные из файла data-logistic.csv.
#Это двумерная выборка, целевая переменная на которой принимает знчения -1 и 1.

data = pandas.read_csv('data-logistic.csv', header = None)
y = data[0]
X = data.iloc[:, 1:]

In [21]:
data.head()

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


In [24]:
Image(url= "imgs/compute-ws.png")

In [25]:
def compute_w1(X, y, w1, w2, k, C):
    l = len(y)
    S = 0
    for i in range(0, l):
        S += y[i] * X[1][i] * (1.0 - 1.0 / (1.0 + math.exp(-y[i] * (w1*X[1][i] + w2*X[2][i]))))
    return w1 + (k * (1.0 / l) * S) - k * C * w1


def compute_w2(X, y, w1, w2, k, C):
    l = len(y)
    S = 0
    for i in range(0, l):
        S += y[i] * X[2][i] * (1.0 - 1.0 / (1.0 + math.exp(-y[i] * (w1*X[1][i] + w2*X[2][i]))))
    return w2 + (k * (1.0 / l) * S) - k * C * w2

In [26]:
def run_grad_descent(X, y, C, w1=0, w2=0, k=0.1, perm_err=1e-5):   # perm_err = допустимая ошибка
    i = 0
    i_max = 10000
    new_w1 = 0 # начальные значения весов
    new_w2 = 0

    while True:
        i += 1
        new_w1 = compute_w1(X, y, w1, w2, k, C)
        new_w2 = compute_w2(X, y, w1, w2, k, C)
        error = math.sqrt((new_w1 - w1)**2 + (new_w2 - w2)**2)
        
        if i >= i_max or error <= perm_err:
            break
        else:
            w1 = new_w1
            w2 = new_w2
            
    return [new_w1, new_w2]

w1, w2 = run_grad_descent(X, y, 0)
regul_w1, regul_w2 = run_grad_descent(X, y, 10)

#Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание.
#В качестве ответа приведите два числа через пробел. 
#Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом.
#Для этого воспользуйтесь сигмоидной функцией: a(x) = 1/(1 + exp(−w1x1 − w2x2)).

def a(X, w1, w2):
    return 1 / (1 + math.exp(-w1*X[1] - w2*X[2]))

y_score = X.apply(lambda x: a(x, w1, w2), axis=1)
ry_score = X.apply(lambda x: a(x, regw1, regw2), axis=1)

auc = roc_auc_score(y, y_score)
r_auc = roc_auc_score(y, ry_score)
auc, r_auc # нужны для оценки качества бинарной классификации

"""
отображает соотношение между долей объектов от общего количества носителей признака, верно классифицированных как 
несущих признак. 

Количественную интерпретацию ROC даёт показатель AUC (англ. area under ROC curve, площадь под ROC-кривой) — площадь,
ограниченная ROC-кривой и осью доли ложных положительных классификаций. Чем выше показатель AUC, тем качественнее
классификатор, при этом значение 0,5 демонстрирует непригодность выбранного метода классификации 
(соответствует случайному гаданию). Значение менее 0,5 говорит, что классификатор действует с точностью до наоборот:
если положительные назвать отрицательными и наоборот, классификатор будет работать лучше. 
"""

(0.9268571428571428, 0.9362857142857142)